In [1]:
# Import Dependencies and Spotipy App

from dateutil.parser import parse as parse_date
import pandas as pd
import spotipy
import spotipy as sp
import spotipy.util as util
import requests
from pprint import pprint
import time
import numpy as np
import pprint

user_id = '1221063701'
client_id = 'a4a8320b97e84b88bc8f4585251085ce'
client_secret = '57f7dc998a714c61b8391858bdc5cc7c'
playlist_id = '0fCFwL8LJE99raQb1g2YvF?si=XS_oL5LhQ9WhD391WFJ4dQ'
redirect_uri='https://www.facebook.com/groups/1223037501082162'


count = 25


token = util.prompt_for_user_token(user_id,
                                   'playlist-read-collaborative',
                                   client_id,
                                   client_secret,
                                   redirect_uri)
sp = spotipy.Spotify(auth=token)

In [2]:
#Read Playlist to pull data from Spotify API
playlist = sp.user_playlist(user_id, playlist_id)
tracks = playlist['tracks']['items']
next_uri = playlist['tracks']['next']
for _ in range(int(playlist['tracks']['total'] / playlist['tracks']['limit'])):
    response = sp._get(next_uri)
    tracks += response['items']
    next_uri = response['next']

tracks_df = pd.DataFrame([(track['track']['id'],
                           track['track']['artists'][0]['name'],
                           track['track']['name'],
                           track['track']['artists'][0]['id'],
                           track['added_by']['id'],
                           parse_date(track['track']['album']['release_date']) if track['track']['album']['release_date'] else None,
                           parse_date(track['added_at']))
                          for track in playlist['tracks']['items']],
                         columns=['id', 'artist', 'name', 'artist_id','user_id', 'release_date', 'added_at'])

In [3]:
# Artist ID in Dataframe

artist_id = tracks_df['artist_id']
artist_id = artist_id.unique().tolist()

# seed_artist_list = seed_artist_list.tolist()

In [4]:
unique_artist = pd.DataFrame(tracks_df['artist_id'].unique())

In [5]:
track_id = tracks_df['id']
track_id = track_id.unique().tolist()

In [6]:
#Pull new artist from artist seed


seed_artist_list = []

for n in range(0,len(artist_id)):
    temp_art = artist_id[n:n+5]
    seed_artist = sp.recommendations(seed_artists = temp_art, limit =count)

    seed_artist = seed_artist['tracks']

    for i in range(len(seed_artist)):
#         print(seed_artist[i]['artists'][0]['id'])
        seed_artist_list.append((artist_id, seed_artist[i]['artists'][0]['id']))
    

In [63]:
artist_id = pd.DataFrame(artist_id)

In [64]:
repeated_artist_ids=pd.DataFrame(np.repeat(unique_artist.values,count))

In [120]:
artist_outputdf = pd.DataFrame(seed_artist_list, repeated_artist_ids)

artist_outputdf = artist_outputdf.reset_index()
artist_outputdf.columns.values[2] = 'Recommended Artist ID'
artist_outputdf.columns.values[1] = 'List'
artist_outputdf.columns.values[0] = "Original Playlist Artist ID"
artist_outputdf = artist_outputdf.drop(columns= 'List')
# artist_outputdf = artist_outputdf.rename({"index":"Original Playlist Artist ID"}, axis='columns') 
artist_outputdf['Original Playlist Artist ID'] = artist_outputdf['Original Playlist Artist ID'].astype(str).str[1:23]
artist_outputdf['Original Playlist Artist ID'] = artist_outputdf['Original Playlist Artist ID'].str.strip("'")

In [121]:
artist_outputdf

,Original Playlist Artist ID,Recommended Artist ID
0,1LZEQNv7sE11VDY3SdxQe,4wrHdTaFAi56czZLNuZlfK
1,1LZEQNv7sE11VDY3SdxQe,4xXCRXOfQKQ2gjWxNhNzYW
2,1LZEQNv7sE11VDY3SdxQe,2eogQKWWoohI3BSnoG7E2U
3,1LZEQNv7sE11VDY3SdxQe,08GQAI4eElDnROBrJRGE0X
4,1LZEQNv7sE11VDY3SdxQe,1iGwHlKgQKtdrtzmlwZ71U
...,...,...
3620,4PJHDzdFoQcklrWU18Qds,4PJHDzdFoQcklrWU18QdsU
3621,4PJHDzdFoQcklrWU18Qds,3vcFXwLEUdfWMu7gTQKyot
3622,4PJHDzdFoQcklrWU18Qds,0s7Ro7Q2XagXEYhWbI9PHv
3623,4PJHDzdFoQcklrWU18Qds,0g5OkLnQogzLrT9s5vw6kl


In [11]:
#Pull new track from track seed

seed_track_list = []

for n in range(0,len(track_id)):
    temp_track = track_id[n:n+5]
    seed_track = sp.recommendations(seed_tracks = temp_track, limit =count)
    
    seed_track = seed_track['tracks']
    
    for i in range(len(seed_track)):
#         print(seed_track[i]['id'])
        seed_track_list.append((seed_track[i]['id']))

In [44]:
track_id = pd.DataFrame(track_id)

In [45]:
repeated_track_ids=pd.DataFrame(np.repeat(track_id.values,count))

In [118]:
outputdf = pd.DataFrame(seed_track_list, repeated_track_ids).drop_duplicates()
outputdf = outputdf.reset_index()
outputdf = outputdf.rename({"index":"Original Playlist Track ID"}, axis='columns') 
outputdf.columns.values[1] = 'Recommended Track ID'
outputdf['Original Playlist Track ID'] = outputdf['Original Playlist Track ID'].astype(str).str[1:23]
outputdf['Original Playlist Track ID'] = outputdf['Original Playlist Track ID'].str.strip("'")

In [119]:
outputdf

,Original Playlist Track ID,Recommended Track ID
0,2cX2coZS1PYBfPs8wgbdW,3FOYOAnJuBqyZ1BYQ1y5gZ
1,2cX2coZS1PYBfPs8wgbdW,5hXEcqQhEjfZdbIZLO8mf2
2,2cX2coZS1PYBfPs8wgbdW,1AIKHVDyU6hCsiLfOlCVxS
3,2cX2coZS1PYBfPs8wgbdW,19Ym5Sg0YyOCa6ao21bdoG
4,2cX2coZS1PYBfPs8wgbdW,1ME3PNQYZbMGN73NN2Vhlh
...,...,...
3195,1ZtNQtpdMtEvSgqL13x5C,2dvFaA5CW1c0YipHNk0EwY
3196,1ZtNQtpdMtEvSgqL13x5C,0CbDc6JeCKQGYVAI89abdm
3197,1ZtNQtpdMtEvSgqL13x5C,3pTA6tD4Xv25EJrAeCjBp6
3198,1ZtNQtpdMtEvSgqL13x5C,6gwizc80zSP5A0kK8dwGNy


In [16]:
#Count # of Tracks

# seed_track_count = len(seed_track_list)
# seed_track_count

In [17]:
#Pull new tracks from seed genre - Only works with specific genres genres. See below



# seed_genre_list = []

# seed_genre = sp.recommendations(seed_genres = ['disco'], limit =15)

# seed_genre = seed_genre['tracks']

# for i in range(len(seed_genre)):
#     print(seed_genre[i]['name'])
#     seed_genre_list.append((seed_genre[i]['name']))
    

In [18]:
# available_seed_genres = [["acoustic",
#     "afrobeat",
#     "alt-rock",
#     "alternative",
#     "ambient"
#     "anime",
#     "black-metal",
#     "bluegrass",
#     "blues",
#     "bossanova",
#     "brazil",
#     "breakbeat",
#     "british",
#     "cantopop",
#     "chicago-house",
#     "children",
#     "chill",
#     "classical",
#     "club",
#     "comedy",
#     "country",
#     "dance",
#     "dancehall",
#     "death-metal",
#     "deep-house",
#     "detroit-techno",
#     "disco",
#     "disney",
#     "drum-and-bass",
#     "dub",
#     "dubstep",
#     "edm",
#     "electro",
#     "electronic",
#     "emo",
#     "folk",
#     "forro",
#     "french",
#     "funk",
#     "garage",
#     "german",
#     "gospel",
#     "goth",
#     "grindcore",
#     "groove",
#     "grunge",
#     "guitar",
#     "happy",
#     "hard-rock",
#     "hardcore",
#     "hardstyle",
#     "heavy-metal",
#     "hip-hop",
#     "holidays",
#     "honky-tonk",
#     "house",
#     "idm",
#     "indian",
#     "indie",
#     "indie-pop",
#     "industrial",
#     "iranian",
#     "j-dance",
#     "j-idol",
#     "j-pop",
#     "j-rock",
#     "jazz",
#     "k-pop",
#     "kids",
#     "latin",
#     "latino",
#     "malay",
#     "mandopop",
#     "metal",
#     "metal-misc",
#     "metalcore",
#     "minimal-techno",
#     "movies",
#     "mpb",
#     "new-age",
#     "new-release",
#     "opera",
#     "pagode",
#     "party",
#     "philippines-opm",
#     "piano",
#     "pop",
#     "pop-film",
#     "post-dubstep",
#     "power-pop",
#     "progressive-house",
#     "psych-rock",
#     "punk",
#     "punk-rock",
#     "r-n-b",
#     "rainy-day",
#     "reggae",
#     "reggaeton",
#     "road-trip",
#     "rock",
#     "rock-n-roll",
#     "rockabilly",
#     "romance",
#     "sad",
#     "salsa",
#     "samba",
#     "sertanejo",
#     "show-tunes",
#     "singer-songwriter",
#     "ska",
#     "sleep",
#     "songwriter",
#     "soul",
#     "soundtracks",
#     "spanish",
#     "study",
#     "summer",
#     "swedish",
#     "synth-pop",
#     "tango",
#     "techno",
#     "trance",
#     "trip-hop",
#     "turkish",
#     "work-out",
#     "world-music"]]


In [19]:
def getTrackFeatures(id):
    meta = sp.track(id)
    features = sp.audio_features(id)

# meta
    name = meta['name']
    album = meta['album']['name']
    artist = meta['album']['artists'][0]['name']
    release_date = meta['album']['release_date']
    length = meta['duration_ms']
    popularity = meta['popularity']
    

# features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    liveness = features[0]['liveness']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    time_signature = features[0]['time_signature']
    valence = features[0]['valence']

    track = [id, name, album, artist, release_date, length, popularity, danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature, valence]
    return track

In [20]:
# loop over track ids for track varriables
tracks = []
for i in range(len(seed_track_list)):
#     time.sleep(.1)
    track = getTrackFeatures(seed_track_list[i])
    tracks.append(track)

# create dataset
song_data = pd.DataFrame(tracks, columns = ['Recommended Track ID','name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence'])

In [ ]:
# loop over track ids for track varriables
tracks = []
for i in range(len(seed_track_list)):
#     time.sleep(.1)
    track = getTrackFeatures(seed_track_list[i])
    tracks.append(track)

# create dataset
song_data = pd.DataFrame(tracks, columns = ['Recommended Track ID','name', 'album', 'artist', 'release_date', 'length', 'popularity', 'danceability', 'acousticness', 'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'time_signature', 'valence'])

In [48]:
output_song_data = song_data

In [49]:
output_song_data = output_song_data.merge(outputdf, how = 'right', on = 'Recommended Track ID').drop_duplicates()

In [67]:
output_artist_data = output_song_data.merge(artist_outputdf, how = 'right', on = "Original Playlist Artist ID").drop_duplicates()

KeyError: 'Original Playlist Artist ID'

In [51]:
output_artist_data.to_csv("reccommended_artist_info.csv")

NameError: name 'output_artist_data' is not defined

In [54]:
output_song_data.to_csv("reccommended_song_info1234.csv", sep = ',')

In [55]:
output_song_data

,Recommended Track ID,name,album,artist,release_date,length,popularity,danceability,acousticness,energy,instrumentalness,liveness,loudness,speechiness,tempo,time_signature,valence,Original Playlist Track ID
0,3FOYOAnJuBqyZ1BYQ1y5gZ,La Tigresa,Joven Navegante,Chicano Batman,2012-06-05,167866,35,0.542,0.0664,0.505,0.792000,0.3050,-7.338,0.0366,125.393,4,0.584,"(2cX2coZS1PYBfPs8wgbdWE,)"
2,5hXEcqQhEjfZdbIZLO8mf2,Everything She Wants,Make It Big,Wham!,1984-10-23,302693,65,0.900,0.3210,0.459,0.003990,0.1650,-15.557,0.0366,115.091,4,0.962,"(2cX2coZS1PYBfPs8wgbdWE,)"
3,1AIKHVDyU6hCsiLfOlCVxS,High Anxiety,A Match & Some Gasoline,The Suicide Machines,2003-06-17,121466,27,0.678,0.0157,0.946,0.000067,0.1470,-3.788,0.0358,102.828,4,0.969,"(2cX2coZS1PYBfPs8wgbdWE,)"
4,19Ym5Sg0YyOCa6ao21bdoG,Gypsy,Greatest Hits,Fleetwood Mac,1988-11-21,264906,72,0.622,0.0170,0.540,0.007710,0.2650,-13.990,0.0261,130.879,4,0.847,"(2cX2coZS1PYBfPs8wgbdWE,)"
6,1ME3PNQYZbMGN73NN2Vhlh,In Your Eyes (feat. Alida),IIII,Robin Schulz,2021-02-26,208222,62,0.570,0.1720,0.672,0.000001,0.0768,-5.888,0.0716,120.018,4,0.461,"(2cX2coZS1PYBfPs8wgbdWE,)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4095,2dvFaA5CW1c0YipHNk0EwY,The Climb,"Directions Home (Songs We Love, Songs You Know)",Point Of Grace,2015-04-07,273280,30,0.466,0.7600,0.318,0.000000,0.2940,-8.909,0.0298,146.082,4,0.357,"(1ZtNQtpdMtEvSgqL13x5Cr,)"
4096,0CbDc6JeCKQGYVAI89abdm,Truth Be Told,Brand New,Matthew West,2020-02-14,219426,63,0.618,0.2910,0.591,0.000000,0.1210,-7.666,0.0364,125.969,4,0.451,"(1ZtNQtpdMtEvSgqL13x5Cr,)"
4097,3pTA6tD4Xv25EJrAeCjBp6,I Promise (Wedding Song),Purified,CeCe Winans,2005-09-13,299333,30,0.426,0.2840,0.520,0.000039,0.0612,-9.258,0.0315,73.344,4,0.506,"(1ZtNQtpdMtEvSgqL13x5Cr,)"
4098,6gwizc80zSP5A0kK8dwGNy,Thank You,Destiny,The Katinas,2001-05-08,238013,40,0.561,0.3860,0.495,0.000000,0.0994,-5.899,0.0263,137.865,4,0.468,"(1ZtNQtpdMtEvSgqL13x5Cr,)"
